# 概率编程语言与库

概率编程语言主要提供概率模型，特别是一般概率网络（如隐马尔科夫模型）的标准描述方法。然后在背后进行自动优化
（如采用theano后端的theano，采用tensorflow后端的edward），模拟（MCMC之类的可以用在几乎所有推断问题上的算法肯定有优化实现）
等，简化了概率建模的工作。

下面summary如下知乎答案

https://www.zhihu.com/question/59442141

## Church

http://projects.csail.mit.edu/church/wiki/Church

是一个概率编程语言，不过好像没什么人用。都搜不出很多信息（和这个坑爹名字可能有关。。）

## WebPPL

http://webppl.org/

看上去用来可视化，教学，演示还不错，对我们统计学内部使用还是另请高明吧。GitHub上才273个star。

## Edward

http://edwardlib.org/

基于tensorflow，看样子功能很多，而且有2335个star在github上。不过其实不如看做tensorflow的库，那个语法。。。

## BUGS

http://www.mrc-bsu.cam.ac.uk/software/bugs/

和Church一样是老牌项目，目测已经没有开发了。

## PYMC3

https://github.com/pymc-devs/pymc3

基于theano，封装比Edward高一点，估计受比它star高好几倍那书加持很大。有2277个star。感觉开发比Edward活跃？

## Stan

http://mc-stan.org/

主推MCMC算法的库，有R，Python与其同名语言的前端与C++的后端（当然这就意味着无法利用GPU加速）。一系列项目但star都没超过1000。
开发感觉也不是很积极。

看起来还是应该在PyMC3和Edward里选，虽然PyMC3有看过那本书的加成和语法上的加分。但听说Edward有30倍速度与更小的内存消耗，
而且有更多的功能。语法虽然蹩脚，但tensorflow本身学习一下也不是没有意义。可以试试Edward，不行再上PyMC3。

In [9]:
import tensorflow as tf
import numpy as np
import edward as ed

In [6]:
x_train = np.load('data/celegans_brain.npy')


In [7]:
from edward.models import Normal, Poisson

N = x_train.shape[0]  # number of data points
K = 3  # latent dimensionality

z = Normal(loc=tf.zeros([N, K]), scale=tf.ones([N, K]))

# Calculate N x N distance matrix.
# 1. Create a vector, [||z_1||^2, ||z_2||^2, ..., ||z_N||^2], and tile
# it to create N identical rows.
xp = tf.tile(tf.reduce_sum(tf.pow(z, 2), 1, keep_dims=True), [1, N])
# 2. Create a N x N matrix where entry (i, j) is ||z_i||^2 + ||z_j||^2
# - 2 z_i^T z_j.
xp = xp + tf.transpose(xp) - 2 * tf.matmul(z, z, transpose_b=True)
# 3. Invert the pairwise distances and make rate along diagonals to
# be close to zero.
xp = 1.0 / tf.sqrt(xp + tf.diag(tf.zeros(N) + 1e3))

x = Poisson(rate=xp)


In [10]:
inference = ed.MAP([z], data={x: x_train})


In [11]:
inference.run(n_iter=2500)


2500/2500 [100%] ██████████████████████████████ Elapsed: 15s | Loss: 35759.867
